<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Store-excel-into-DataFrame" data-toc-modified-id="Store-excel-into-DataFrame-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Store excel into DataFrame</a></span></li><li><span><a href="#Create-new-data-with-select-columns" data-toc-modified-id="Create-new-data-with-select-columns-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Create new data with select columns</a></span></li><li><span><a href="#Store-JSON-data-into-a-DataFrame" data-toc-modified-id="Store-JSON-data-into-a-DataFrame-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Store JSON data into a DataFrame</a></span></li><li><span><a href="#Clean-DataFrame" data-toc-modified-id="Clean-DataFrame-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Clean DataFrame</a></span></li><li><span><a href="#Connect-to-local-database" data-toc-modified-id="Connect-to-local-database-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Connect to local database</a></span></li><li><span><a href="#Check-for-tables" data-toc-modified-id="Check-for-tables-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Check for tables</a></span></li><li><span><a href="#Use-pandas-to-load-csv-converted-DataFrame-into-database" data-toc-modified-id="Use-pandas-to-load-csv-converted-DataFrame-into-database-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Use pandas to load csv converted DataFrame into database</a></span></li><li><span><a href="#Use-pandas-to-load-json-converted-DataFrame-into-database" data-toc-modified-id="Use-pandas-to-load-json-converted-DataFrame-into-database-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Use pandas to load json converted DataFrame into database</a></span></li><li><span><a href="#Confirm-data-has-been-added-by-querying-the-customer_name-table" data-toc-modified-id="Confirm-data-has-been-added-by-querying-the-customer_name-table-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Confirm data has been added by querying the customer_name table</a></span></li><li><span><a href="#Confirm-data-has-been-added-by-querying-the-customer_location-table" data-toc-modified-id="Confirm-data-has-been-added-by-querying-the-customer_location-table-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Confirm data has been added by querying the customer_location table</a></span></li></ul></div>

In [46]:
import pandas as pd
from sqlalchemy import create_engine

### Store excel into DataFrame

In [52]:
#csv_file = "../Resources/customer_data.csv"
excel_file = "Resources/shipmentstostate2018.xls"
xls = pd.ExcelFile(excel_file)
sheet_names = xls.sheet_names
#customer_data_df = pd.read_csv(csv_file)
for ThisSheet in sheet_names:
    print(ThisSheet)
    DfForShipMant = pd.read_excel(excel_file, sheet_name = ThisSheet)
    DfForShipMant.dropna(inplace= True)
    headers = DfForShipMant.iloc[0]
    new_df  = pd.DataFrame(DfForShipMant.values[1:], columns=headers)
    new_df['month']=ThisSheet
    new_df['year']=2018
    new_df.head()
    new_df.tail()
headers

#customer_data_df.head()

#shipmementstostate2018.head()



January
February
March
April
May
June
July
August
September
October
November
December


Manufactured Housing              State
Unnamed: 1              Single- Section
Unnamed: 2               Multi- Section
Unnamed: 3                        Total
Unnamed: 4                       Floors
Name: 3, dtype: object

### Create new data with select columns

In [40]:
    
# copy the 'id', 'State', 'Single-Section','Multi-Section' to new data frame
new_ship_data2018 = new_df[['State','Single- Section' ,'Multi- Section']].copy()
new_ship_data2018['year']=2018
new_ship_data2018.head()

3,State,Single- Section,Multi- Section,year
0,Alabama,660,127,2018
1,Alaska,3,0,2018
2,Arizona,28,114,2018
3,Arkansas,93,70,2018
4,California,68,277,2018


### Store JSON data into a DataFrame

In [ ]:
json_file = "../Resources/customer_location.json"
customer_location_df = pd.read_json(json_file)
customer_location_df.head()

### Clean DataFrame

In [ ]:
new_customer_location_df = customer_location_df[["id", "address", "us_state"]].copy()
new_customer_location_df.head()

### Connect to local database

In [ ]:
import pg
conn = pg.DB(host="localhost", user="postgres", passwd="G@dhav7865l", dbname="customer_db", port=63655)

#rds_connection_string = "postgres:G\@dhav7865l@localhost:63655/customer_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables

In [ ]:
engine.table_names()

### Use pandas to load csv converted DataFrame into database

In [ ]:
new_customer_data_df.to_sql(name='customer_name', con=engine, if_exists='append', index=False)

### Use pandas to load json converted DataFrame into database

In [ ]:
new_customer_location_df.to_sql(name='customer_location', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the customer_name table
* NOTE: can also check using pgAdmin

In [ ]:
pd.read_sql_query('select * from customer_name', con=engine).head()

### Confirm data has been added by querying the customer_location table

In [ ]:
pd.read_sql_query('select * from customer_location', con=engine).tail()